<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT license.</i>

# MoRec: A Data-Centric Multi-Objective Learning Framework for Responsible Recommendation Systems

MoRec[[1]](https://arxiv.org/abs/2310.13260v1) is a data-centric multi-objective framework designed for responsible recommendation systems. Concretely, MoRec adopts a tri-level framework to optimize diverse objectives simultaneously, comprising a PID-based objective coordinator for trade-off among objectives and an adaptive data sampler for unified objective modeling. 


## Strengths of MoRec
- MoRec is model-agnostic, which is capable of converting an accuracy-oriented model to multi-objective model
- MoRec adopts a post-training strategy, which is able to convert a well-trained model to multi-objective model at a low cost
- MoRec exhibit great capability in objective controlling, which could optimize model with objective preference without sacrificing too much accuracy

## Data requirements

MoRec is capable of optimizing accuracy, revenue, fairness and alignment objectives simultaneously. 

- For accuracy, basical user-item interaction files are required, including `train.csv`, `valid.csv`, `test.csv` and `user_history.csv`. 
  `train.csv`, `valid.csv`, `test.csv` represent interactions in training set, validation set and test set respectively, which are formatted as follows:

  <table>
      <tr>
          <td>user_id</td>
          <td>item_id</td>
      </tr>
      <tr>
          <td>1</td>
          <td>1</td>
      </tr>
      <tr>
          <td>1</td>
          <td>2</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
      </tr>
      <tr>
          <td>100</td>
          <td>254</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
      </tr>
  </table>

  `user_history.csv` represents the user's interaction history, consist of interactions in training set and validation set, which is formatted as follows:

  <table>
      <tr>
          <td>user_id</td>
          <td>item_seq</td>
      </tr>
      <tr>
          <td>1</td>
          <td>1,2,3,...</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
      </tr>
      <tr>
          <td>100</td>
          <td>254,257,327,...</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
      </tr>
  </table>

- For revenue, MoRec would sample data samples according to their weights, i.e. item price. For fairness, MoRec aims to improve the accuracy preformance of the most disadvantaged group. For alignment, MoRec targets on aligning the model's distribution with some pre-defined expectation distribution. To model those objectives, `item_meta_morec_filename` is required to provide item weights, fairness group and alignment group. And if you want to set the pre-set distribution for alignment,  `align_dist_filename` is needed. By default, the expected distribution to aligned with is the distribution derived from the training set. Here are the example of item_meta_morec_file  and align_dist_file.
  
  - item_meta_morec_file: `item_meta_morec.csv`, columns separated by comma

  <table>
      <tr>
          <td>item_id</td>
          <td>weight</td>
          <td>fair_group</td>
          <td>align_group</td>
      </tr>
      <tr>
          <td>1</td>
          <td>2.35</td>
          <td>1</td>
          <td>2</td>
      </tr>
      <tr>
          <td>2</td>
          <td>63.21</td>
          <td>5</td>
          <td>1</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
          <td>...</td>
          <td>...</td>
      </tr>
      <tr>
          <td>100</td>
          <td>5.89</td>
          <td>5</td>
          <td>4</td>
      </tr>
      <tr>
          <td>...</td>
          <td>...</td>
          <td>...</td>
          <td>...</td>
      </tr>
  </table>

  - align_dist_file: `expected_align_dist.csv`, columns separated by comma

  <table>
      <tr>
          <td>group_id</td>
          <td>proportion</td>
      </tr>
      <tr>
          <td>1</td>
          <td>0.21</td>
      </tr>
      <tr>
          <td>2</td>
          <td>0.12</td>
      </tr>
      <tr>
          <td>3</td>
          <td>0.33</td>
      </tr>
      <tr>
          <td>4</td>
          <td>0.22</td>
      </tr>
      <tr>
          <td>5</td>
          <td>0.12</td>
      </tr>
</table>


### Example: Amazon Electronics dataset

#### Data Preparation

We put the script for downloading and preprocessing ml-100k into the our [example folder](../../preprocess/download_split_ml100k.py). Here we would call the functions defined in the script. The preprocessed csv files would be saved in `~/.unirec/dataset/ml-100k`. 

We believe that you could easily process your own dataset to obtain `train.csv`, `valid.csv`, `test.csv` and `user_history.csv` using leave-one-out strategy. 

As for the columns in `item_meta_morec.csv` file, we set price of items as weight and group items according to their categories as the fair_group. As for align_group, we divide items according to the popularity, where items with similar popularity are put into the same group.

In [1]:
import os
import sys
import shutil
import datetime
from copy import deepcopy

sys.path.append(os.path.abspath("../../"))
from preprocess.download_split_amazon import preprocess_amazon
from preprocess.prepare_data import process_transaction_dataset

import unirec
from unirec.main import main
from unirec.constants.protocols import DataFileFormat

UNIREC_PATH = os.path.dirname(unirec.__file__)

In [2]:
UNIREC_PATH

'/anaconda/envs/unirec/lib/python3.9/site-packages/unirec'

In [3]:
preprocess_amazon("Electronics")

original dataset size: (7824482, 9)
filter by rating>3 dataset size: (5833322, 9)
drop_duplicates dataset size: (5833322, 9)
Ite: 0, users: 177149 / 3256144, items: 51997 / 410110
Ite: 1, users: 130234 / 177149, items: 45697 / 51997
Ite: 2, users: 125581 / 130234, items: 44973 / 45697
Ite: 3, users: 125011 / 125581, items: 44866 / 44973
Ite: 4, users: 124916 / 125011, items: 44847 / 44866
k-core filtered dataset size: (1072840, 9)
#Users: 124916, #Items: 44847
size in Train/Valid/Test: (823008, 2) / (124916, 2) / (124916, 2)
0 raws price not float
44847/44847 item prices existing in meta df
Map dict saved in /home/v-huangxu/.unirec/dataset/Electronics/map.json.
Item meta file saved in /home/v-huangxu/.unirec/dataset/Electronics/item_meta_morec.csv.


##### Binary Data File Preparation

Upon the interaction files are processed, UniRec requires to convert them into binary files for time-saving loading. We provide the tools in [example folder](../../preprocess/prepare_data.py) to easily obtain the pickle file.

Note that the function `process_transaction_dataset` requires some meta information of the csv files, such as the directory path, the seperator, header , file format and so on. 

Note that we have defined several data formats in UniRec, you can list all formats using codes below.

In [4]:
# All supported data file formats
for format in DataFileFormat.__members__.values():
    print(f"{format}: {format.value}")

DataFileFormat.T1: user-item
DataFileFormat.T2: user-item-label
DataFileFormat.T2_1: user-item-label-session
DataFileFormat.T3: user-item-rating
DataFileFormat.T4: user-item_group-label_group
DataFileFormat.T5: user-item_seq
DataFileFormat.T5_1: user_item_seq
DataFileFormat.T6: user-item_seq-time_seq
DataFileFormat.T7: label-index_group-value_group


In [5]:
binary_data_folder_path = os.path.expanduser("~/.unirec/dataset/binary/")

UNIREC_PATH = os.path.dirname(unirec.__file__)

BINARY_FILE_CONFIG = {
    "raw_datapath": os.path.expanduser("~/.unirec/dataset/Electronics"), # the dir of csv files
    "outpathroot": binary_data_folder_path, # the output dir of processed binary files
    "dataset_name": "Electronics", # the dataset name, set as you like
    "example_yaml_file": os.path.join(UNIREC_PATH, "config/dataset/example.yaml"), # Do not modify the value
    "index_by_zero": 0,  # whether the user_id and item_id start from 0
    "sep": "\t" ,   # the seperator of csv files 
    "train_file": 'train.csv',  # the filename of training csv file
    "train_file_format": 'user-item', 
    "train_file_has_header": 1, # whether the training file has header
    "train_file_col_names": "['user_id', 'item_id']",  # the columns of training csv file
    "train_neg_k": 0,  
    "valid_file": 'valid.csv', # the filename of validation csv file
    "valid_file_format": 'user-item', 
    "valid_file_has_header": 1, # whether the validation file has header
    "valid_file_col_names": "['user_id', 'item_id']", # the columns of validation csv file
    "valid_neg_k": 0, 
    "test_file": 'test.csv', # the filename of test csv file
    "test_file_format": 'user-item', 
    "test_file_has_header": 1, # whether the test file has header
    "test_file_col_names": "['user_id', 'item_id']", # the columns of test csv file
    "test_neg_k": 0, 
    "user_history_file": 'user_history.csv', # the filename of history csv file
    "user_history_file_format": 'user-item_seq', 
    "user_history_file_has_header": 1, # whether the history file has header
    "user_history_file_col_names": "['user_id', 'item_seq']" # the columns of history csv file
}

In [6]:
process_transaction_dataset(BINARY_FILE_CONFIG)    # the binary files would be saved in `binary_data_folder_path`

data shape of train.csv is (823008, 2)
data dtypes is user_id    int64
item_id    int64
dtype: object
saving train.pkl at 26/10/2023 17:29:16
finish saving train.pkl at 26/10/2023 17:29:16
In saving:
   user_id  item_id
0        1        1
1        1        2
2        1        3
3        1        4
4        2        7
data.shape=(823008, 2)

data shape of valid.csv is (124916, 2)
data dtypes is user_id    int64
item_id    int64
dtype: object
saving valid.pkl at 26/10/2023 17:29:16
finish saving valid.pkl at 26/10/2023 17:29:16
In saving:
   user_id  item_id
0        1        5
1        2       10
2        3       17
3        4       23
4        5       29
data.shape=(124916, 2)

data shape of test.csv is (124916, 2)
data dtypes is user_id    int64
item_id    int64
dtype: object
saving test.pkl at 26/10/2023 17:29:16
finish saving test.pkl at 26/10/2023 17:29:16
In saving:
   user_id  item_id
0        1        6
1        2       11
2        3       18
3        4       24
4        5     

In [7]:
# for the item_meta_morec.csv file, we copy it to the binary file path as well
shutil.copyfile(os.path.join(BINARY_FILE_CONFIG['raw_datapath'], 'item_meta_morec.csv'), os.path.join(BINARY_FILE_CONFIG['outpathroot'], BINARY_FILE_CONFIG['dataset_name'], 'item_meta_morec.csv'))

'/home/v-huangxu/.unirec/dataset/binary/Electronics/item_meta_morec.csv'

#### MoRec pretraining stage: accuracy-oriented model training

Since MoRec provides a post-training strategy to convert a single-objective model (usually an accuracy-oriented model) to a multi-objective model, we need to train the accuracy-oriented model first.

1. First, setup morec_configurations, including hyperparameters, file paths.
2. Second, training with unirec's user-friendly interface.

In [8]:
dataset = "Electronics"
model = "MF"
ckpt_output_path = os.path.expanduser("~/.unirec/output")

GLOBAL_CONF = {
    'config_dir': f"{os.path.join(UNIREC_PATH, 'config')}",
    'exp_name': '',
    'checkpoint_dir': datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
    'model': model,
    'dataloader': 'BaseDataset',
    'dataset': dataset,
    'dataset_path': os.path.join(BINARY_FILE_CONFIG['outpathroot'], dataset),
    'output_path': os.path.join(ckpt_output_path, dataset, model),
    'learning_rate': 0.001,
    'scheduler': None,
    'dropout_prob': 0.0,
    'embedding_size': 64,
    'user_pre_item_emb': 0,
    'loss_type': 'bpr',
    'max_seq_len': 10,
    'has_user_bias': 0,
    'has_item_bias': 0,
    'epochs': 200,
    'early_stop': 10,
    'batch_size': 512,
    'valid_batch_size': 1024,
    'n_sample_neg_train': 10,
    'valid_protocol': 'one_vs_all',
    'test_protocol': 'one_vs_all',
    'grad_clip_value': -1,
    'weight_decay': 1e-6,
    'history_mask_mode': 'autoagressive',
    'user_history_filename': "user_history",
    'metrics': "['hit@10', 'rhit@10', 'pop-kl@10', 'least-misery']",
    'key_metric': "hit@10",
    'num_workers': 4,
    'num_workers_test': 0,
    'verbose': 2,
    'neg_by_pop_alpha': 1.0,
    'item_meta_morec_filename': 'item_meta_morec.csv',
    'align_dist_filename': None,  # the expected alignment distribution is set as the distribution derived from the training set
    'use_tensorboard': 1
}

In [9]:

for arg in sys.argv:  # arguments conflict in notebooks, this is only required in notebooks
    if "-f" in arg:
        sys.argv.remove(arg)

pretrain_config = deepcopy(GLOBAL_CONF)

pretrain_config['checkpoint_dir'] = 'morec_pretrain_' + pretrain_config['checkpoint_dir']
pretrain_config['exp_name'] = "MoRec-Pretrain"

pretrain_result = main.run(pretrain_config)

print(pretrain_result)

Load configuration files from /anaconda/envs/unirec/lib/python3.9/site-packages/unirec/config


[INFO] MF-MoRec-Pretrain: config={'gpu_id': 0, 'use_gpu': True, 'seed': 2022, 'state': 'INFO', 'verbose': 2, 'saved': True, 'use_tensorboard': 1, 'use_wandb': False, 'init_method': 'normal', 'init_std': 0.02, 'init_mean': 0.0, 'scheduler': None, 'scheduler_factor': 0.1, 'time_seq': 0, 'seq_last': False, 'has_user_emb': True, 'has_user_bias': 0, 'has_item_bias': 0, 'use_features': False, 'use_text_emb': False, 'use_position_emb': True, 'load_pretrained_model': False, 'embedding_size': 64, 'hidden_size': 128, 'inner_size': 128, 'dropout_prob': 0.0, 'epochs': 200, 'batch_size': 512, 'learning_rate': 0.001, 'optimizer': 'adam', 'eval_step': 1, 'early_stop': 10, 'clip_grad_norm': None, 'weight_decay': 1e-06, 'num_workers': 4, 'persistent_workers': False, 'pin_memory': False, 'shuffle_train': False, 'use_pre_item_emb': 0, 'loss_type': 'bpr', 'ccl_w': 150, 'ccl_m': 0.4, 'distance_type': 'dot', 'metrics': "['hit@10', 'rhit@10', 'pop-kl@10', 'least-misery']", 'key_metric': 'hit@10', 'test_proto

Writing logs to /home/v-huangxu/.unirec/output/Electronics/MF/MF-MoRec-Pretrain.2023-10-26_172919.100.txt


[INFO] MF-MoRec-Pretrain: Done. 124917 of users have history.
[INFO] MF-MoRec-Pretrain: Constructing dataset of task type: train
[DEBUG] MF-MoRec-Pretrain: loading train at 26/10/2023 17:29:21
[DEBUG] MF-MoRec-Pretrain: Finished loading train at 26/10/2023 17:29:21
[INFO] MF-MoRec-Pretrain: Finished initializing <class 'unirec.data.dataset.basedataset.BaseDataset'>
[INFO] MF-MoRec-Pretrain: Constructing dataset of task type: valid
[DEBUG] MF-MoRec-Pretrain: loading valid at 26/10/2023 17:29:21
[DEBUG] MF-MoRec-Pretrain: Finished loading valid at 26/10/2023 17:29:21
[INFO] MF-MoRec-Pretrain: Finished initializing <class 'unirec.data.dataset.basedataset.BaseDataset'>
[INFO] MF-MoRec-Pretrain: MF(
  (scorer_layers): InnerProductScorer()
  (user_embedding): Embedding(124917, 64, padding_idx=0)
  (item_embedding): Embedding(44848, 64, padding_idx=0)
)
Trainable parameter number: 10864960
All trainable parameters:
user_embedding.weight : torch.Size([124917, 64])
item_embedding.weight : torch

Logger close successfully.
{'hit@10': 0.016306958275961445, 'rhit@10': 1.4515302283134264, 'pop-kl@10': 1.4378998481908087, 'min-hit@10': 0.093349071380622063, 'min-rhit@10': 0.2596570821212799}


#### MoRec fine-tuning stage: multi-objective model tuning

In this stage, the pretrained model is loaded and then trained successively toward a multi-objective model. 

Here we only need to set parameters for MoRec. There are several important arguments here.

- enable_morec: to enable MoRec finetuning
- model_file: the checkpoint file path of pretrained model, which is set as the output path of pretraining stage
- morec_objectives: the objectives to be optimized in MoRec
- morec_ngroup: group items according to weight. If -1, not use group
- morec_alpha: the learning rate to update sampling weight with signed SGD in MoRec data sampler
- morec_lambda: the coef $\lambda$ used in loss synthesis
- morec_expect_loss: expected loss for accuracy, used in PID-based objective coordinatoor
- morec_beta_min, morec_beta_max: the range of output of PID
- morec_K_p, morec_K_i: the coef of proportional and integral part of PID
- morec_objective_controller: MoRec objective coordinator(controller), optional ["Static", "PID"]
- morec_objective_weights: Weight to set objective preference. For PID controller, the length should be equal to the number of objectives except accuracy. For Static controller, the length should be the number of all objectives including accuracy and the last weight is for accuracy.
- early_stop: early_stop epochs. In multi-objective settings, we could not set early stopping by minitoring one objective, so we set to -1 to disable early stopping.

In [10]:
# MoRec multi-objective post-training (fine-tuning) stage 
morec_config = deepcopy(GLOBAL_CONF)

morec_config['enable_morec'] = 1
morec_config['exp_name'] = 'Morec-Finetune'

# pretrained model file is loaded by the `model_file` argument
morec_config['model_file'] = os.path.join(pretrain_config['output_path'], pretrain_config['checkpoint_dir'], f"{pretrain_config['model']}-{pretrain_config['exp_name']}.pth")
morec_config['checkpoint_dir'] = "morec_finetune_" + morec_config['checkpoint_dir']

# MoRec parameters
morec_config['morec_objectives']=['fairness', 'alignment', 'revenue']
morec_config["morec_ngroup"] = 40
morec_config["morec_alpha"] = 0.1
morec_config["morec_lambda"] = 0.2
morec_config["morec_expect_loss"] = 0.20
morec_config["morec_beta_min"] = 0.1
morec_config["morec_beta_max"] = 1.5
morec_config["morec_K_p"] = 0.05
morec_config["morec_K_i"] = 0.001
morec_config["morec_objective_controller"] = "PID"
morec_config["morec_objective_weights"] = "[0.2,0.2,0.6]"

morec_config["epochs"] = 30
morec_config["early_stop"] = -1

morec_result = main.run(morec_config)

[INFO] MF-Morec-Finetune: config={'gpu_id': 0, 'use_gpu': True, 'seed': 2022, 'state': 'INFO', 'verbose': 2, 'saved': True, 'use_tensorboard': 1, 'use_wandb': False, 'init_method': 'normal', 'init_std': 0.02, 'init_mean': 0.0, 'scheduler': None, 'scheduler_factor': 0.1, 'time_seq': 0, 'seq_last': False, 'has_user_emb': True, 'has_user_bias': 0, 'has_item_bias': 0, 'use_features': False, 'use_text_emb': False, 'use_position_emb': True, 'load_pretrained_model': False, 'embedding_size': 64, 'hidden_size': 128, 'inner_size': 128, 'dropout_prob': 0.0, 'epochs': 30, 'batch_size': 512, 'learning_rate': 0.001, 'optimizer': 'adam', 'eval_step': 1, 'early_stop': -1, 'clip_grad_norm': None, 'weight_decay': 1e-06, 'num_workers': 4, 'persistent_workers': False, 'pin_memory': False, 'shuffle_train': False, 'use_pre_item_emb': 0, 'loss_type': 'bpr', 'ccl_w': 150, 'ccl_m': 0.4, 'distance_type': 'dot', 'metrics': "['hit@10', 'rhit@10', 'pop-kl@10', 'least-misery']", 'key_metric': 'hit@10', 'test_protoc

Load configuration files from /anaconda/envs/unirec/lib/python3.9/site-packages/unirec/config
Writing logs to /home/v-huangxu/.unirec/output/Electronics/MF/MF-Morec-Finetune.2023-10-26_201035.78.txt


[INFO] MF-Morec-Finetune: Done. 124917 of users have history.
[INFO] MF-Morec-Finetune: Loading model from checkpoint: /home/v-huangxu/.unirec/output/Electronics/MF/morec_pretrain_2023-10-26_17-29-19/MF-MoRec-Pretrain.pth ...
[INFO] MF-Morec-Finetune: Constructing dataset of task type: train
[DEBUG] MF-Morec-Finetune: loading train at 26/10/2023 20:10:36
[DEBUG] MF-Morec-Finetune: Finished loading train at 26/10/2023 20:10:36
[INFO] MF-Morec-Finetune: Finished initializing <class 'unirec.data.dataset.basedataset.BaseDataset'>
[INFO] MF-Morec-Finetune: Constructing dataset of task type: train
[DEBUG] MF-Morec-Finetune: loading valid at 26/10/2023 20:10:37
[DEBUG] MF-Morec-Finetune: Finished loading valid at 26/10/2023 20:10:37
[INFO] MF-Morec-Finetune: Finished initializing <class 'unirec.data.dataset.basedataset.BaseDataset'>
[INFO] MF-Morec-Finetune: Constructing dataset of task type: valid
[DEBUG] MF-Morec-Finetune: loading valid at 26/10/2023 20:10:37
[DEBUG] MF-Morec-Finetune: Fini

static weight: [0.2, 0.2, 0.6].


Evaluate: 100%|██████████| 122/122 [01:21<00:00,  1.50it/s]
[INFO] MF-Morec-Finetune: epoch 0 evaluating [time: 81.21s, hit@10: 0.028491]
[INFO] MF-Morec-Finetune: complete scores on valid set: 
hit@10:0.0284911460501457 min-hit@10:0.017619420516836334 min-rhit@10:0.602854552049329 pop-kl@10:1.43916574692065 rhit@10:2.5915429568670145
[INFO] MF-Morec-Finetune: Saving best model at epoch 0 to /home/v-huangxu/.unirec/output/Electronics/MF/morec_finetune_2023-10-26_17-29-19/MF-Morec-Finetune.pth
[INFO] MF-Morec-Finetune: 
>> epoch 1
Train: 100%|██████████| 1608/1608 [00:51<00:00, 31.52it/s]
[INFO] MF-Morec-Finetune: epoch 1 training [time: 51.02s, train loss: 197.5243]
[INFO] MF-Morec-Finetune: one_vs_all
Evaluate: 100%|██████████| 122/122 [01:23<00:00,  1.47it/s]
[INFO] MF-Morec-Finetune: epoch 1 evaluating [time: 83.10s, hit@10: 0.025841]
[INFO] MF-Morec-Finetune: complete scores on valid set: 
hit@10:0.025841365397547153 min-hit@10:0.00797968806673921 min-rhit@10:0.36295973884657234 po

Logger close successfully.


#### Performance Comparisons

The MoRec framework could improve model's performance in rhit@10, pop-kl@10, min-hit@10, which represent revenue, alignment and fairness respectively. And the improvements only sacrificy little accuracy, resulting in a 2.08% relative drop in term of hit@10. 

Note, the details of metrics used here are given in our [paper](https://arxiv.org/abs/2310.13260v1). The higher metrics represent the better performance, except the pop-kl.


<table>
    <tr>
        <td></td>
        <td>Accuracy (hit@10)</td>
        <td>Revenue (rhit@10)</td>
        <td>Alignment (pop-kl@10)</td>
        <td>Fairness(min-hit@10)</td>
    </tr>
    <tr>
        <td>Pretrain</td>
        <td>0.01630</td>
        <td>1.4515</td>
        <td>1.4379</td>
        <td>0.09335</td>
    </tr>
    <tr>
        <td>MoRec</td>
        <td>0.01705</td>
        <td>2.0424</td>
        <td>0.4071</td>
        <td>0.01052</td>
    </tr>
</table>


In [11]:
print("Pretrain: ", pretrain_result)
print("Finetune: ", morec_result)

Pretrain:  {'hit@10': 0.016306958275961445, 'rhit@10': 1.4515302283134264, 'pop-kl@10': 1.4378998481908087, 'min-hit@10': 0.093349071380622063, 'min-rhit@10': 0.2596570821212799}
Finetune:  {'hit@10': 0.017051458580165872, 'rhit@10': 2.04236959236607, 'pop-kl@10': 0.40707595507603583, 'min-hit@10': 0.010516894159767285, 'min-rhit@10': 0.38793632874237977}
